In [ ]:
from pypdf import PdfReader

reader = PdfReader("klima-update.pdf")
def read_pages(reader, pages):
    text = ""
    for page in pages:
        pdf_page = reader.pages[page]
        text += pdf_page.extract_text(extraction_mode="layout", layout_mode_strip_rotated=False)
    return text

print(read_pages(reader, list(range(2, 14))))

In [ ]:
from langchain_openai import AzureChatOpenAI

client = AzureChatOpenAI(
  openai_api_version="2024-02-01",
  deployment_name="...",
  azure_endpoint="...",
  api_key="...",
  model_name="gpt-4o"
)

In [73]:
from typing import List
from pydantic import BaseModel, Field

class MeasureUpdateForecast(BaseModel):
    year: int
    value: float

class MeasureUpdateData(BaseModel):
    title: str
    text_updates: List[str]
    forecasts: List[MeasureUpdateForecast]
    forecasts_of_costs: List[MeasureUpdateForecast]

class MeasureUpdateDataList(BaseModel):
    updates: List[MeasureUpdateData]

In [86]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are an expert extraction algorithm. "
            "Only extract relevant information from the text. "
            "If you do not know the value of an attribute asked to extract, "
            "return null for the attribute's value."
            "If there are no forecasts or forecasts of costs, return an empty list for the attribute's value."
            "Ensure that the years for the forecasts are correct.",
        ),
        # Please see the how-to about improving performance with
        # reference examples.
        # MessagesPlaceholder('examples'),
        ("human", "{text}"),
    ]
)

runnable = prompt | client.with_structured_output(schema=MeasureUpdateDataList)


In [87]:
energieerzeugung_text = read_pages(reader, list(range(2, 14)))


In [88]:
results = runnable.invoke({"text": energieerzeugung_text})

In [ ]:
print(results.updates)